## Advanced Semantic Search with Sentence Transformers and MLflow

Welcome to this hands-on tutorial where we will be exploring **Advanced Semantic Search using Sentence Transformers** and **MLflow**. This guide is designed for those who are keen on diving deeper into the realms of Natural Language Processing (NLP), with a special focus on the practical integration of sophisticated NLP models with MLflow for robust model management.

### Understanding Semantic Search

Semantic search refers to the process of searching not just based on keywords, but by understanding the intent and contextual meaning of the search query. Unlike traditional search algorithms that rely on keyword matching, semantic search algorithms interpret the nuances and semantics of language to find results that are contextually relevant to the query. This approach mirrors how humans understand language, considering the varied and subtle meanings words can have in different contexts.

### Harnessing Power of Sentence Transformers for Search

**Sentence Transformers** are a unique offshoot of the transformer models, fine-tuned to generate **contextually rich** sentence embeddings. These embeddings are vector representations of sentences that capture their semantic meaning. Using Sentence Transformers, we can convert both our search queries and our database (or corpus) of text into these embeddings. By comparing the embeddings of the query with those in the corpus, we can identify the most semantically similar entries - a core principle of semantic search.

Originating from the acclaimed Transformers library by 🤗 Hugging Face, these models are adept at tasks like semantic search and clustering. In this tutorial, we'll see how `sentence-transformers` are employed not just for their technical prowess, but also for their ability to sift through a corpus sprinkled with humor and wit.

### MLflow: A Vanguard in Model Management and Deployment

MLflow's integration into this process is twofold. It not only streamlines our project management but also brings to the table a customizable environment for NLP models:

- **Efficient Experiment Logging**: Track your NLP experiments with ease, logging intricate details like model parameters and embeddings using MLflow.
- **Custom `PythonModel` in Action**: We'll delve into a bespoke `PythonModel` implementation that transcends standard outputs, returning results that are not only accurate but also humorously resonant.
- **Lifecycle Management Made Simple**: Navigate through the complexities of NLP model development with MLflow's versioning and configuration control.
- **Seamless Deployment and Reproducibility**: Ready your models for real-world application with the assurance of reproducibility and consistency, courtesy of MLflow.

### What You Will Learn

In this tutorial, you will:

- Implement and manipulate the `sentence-transformers` library for advanced semantic search.
- Explore and customize MLflow’s `PythonModel` to meet unique project needs.
- Manage and log models and configurations within the MLflow ecosystem.
- Deploy sophisticated models for practical use, leveraging MLflow's robust deployment features.

By the end of this tutorial, not only will you have a firmer grasp on conducting intricate semantic searches using Sentence Transformers, but you'll also gain invaluable insights into the versatility of MLflow for model deployment and management. Get ready to embark on a journey that combines the rigor of NLP with a useful application in document retrieval, enhancing your understanding of both the sentence transformers library and MLflow.

## Understanding the Semantic Search Model with MLflow and Sentence Transformers

In this section, we delve into the technicalities of our `SemanticSearchModel`, a custom model leveraging MLflow and the Sentence Transformers library. This model is tailored for a semantic search use case, specifically designed for searching through blog posts based on user queries.

### MLflow and Custom `pyfunc` Models

MLflow is a versatile platform for managing the machine learning lifecycle, including experimentation, reproducibility, and deployment. One of its powerful features is the ability to define custom Python function (`pyfunc`) models. These models provide a flexible way to encapsulate your logic and can be easily deployed and managed within the MLflow ecosystem.

In our example, we define the `SemanticSearchModel`, a subclass of `PythonModel`. This class encapsulates the entire logic needed for our semantic search application, making it a self-contained, deployable artifact. By leveraging MLflow's `pyfunc`, we gain the following advantages:

- **Flexibility in Model Definition**: Custom logic for semantic search is neatly wrapped in a class, providing clear structure and readability.
- **Ease of Deployment**: The model can be deployed as a standalone service or integrated into larger applications, with MLflow handling the complexities of deployment.
- **Version Control and Experiment Tracking**: MLflow allows us to track different versions of our model and experiment with various parameters seamlessly.

### The Model's Core Functionalities

#### Context Loading

The `load_context` method is responsible for loading the model and the corpus used for semantic search. This method:

- Loads a pre-trained Sentence Transformer model, which is responsible for converting text into semantically meaningful embeddings.
- Reads a corpus of blog posts from a file, providing the content against which user queries will be compared.

#### Predict Method

The `predict` method is the heart of our semantic search functionality. Here's a breakdown of its key components:

- **Input Validation**: It ensures the input, whether in DataFrame or dictionary format, is valid and extracts the query sentence.
- **Query Encoding**: The input query is encoded into an embedding using the Sentence Transformer model.
- **Cosine Similarity Computation**: The method calculates the cosine similarity between the query embedding and the corpus embeddings. This similarity score determines how relevant each corpus entry is to the query.
- **Top Results Extraction**: The `top_k` most similar entries are extracted. This parameter is flexible and can be adjusted without needing to redeploy the model.
- **Relevancy Filtering**: A `minimum_relevancy` parameter is introduced, allowing the user to filter out results below a certain relevancy threshold. This enhances the model's usability by ensuring only sufficiently relevant results are returned.
- **Warning Mechanism**: If all top results are below the `minimum_relevancy` threshold, a warning is issued, and the most relevant result is returned. This ensures the model always provides a result, enhancing user experience.

### Conclusion

Through this implementation, we demonstrate how MLflow and Sentence Transformers can be combined to create a robust, flexible, and user-friendly semantic search model. This model showcases the power of NLP in practical applications and exemplifies how MLflow's framework can accommodate complex logic and user experience considerations, making it an invaluable tool in modern machine learning workflows.


In [1]:
import mlflow
from mlflow.pyfunc import PythonModel
from mlflow.models.signature import infer_signature
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from typing import List
import warnings


class SemanticSearchModel(PythonModel):
    def load_context(self, context):
        """Load the model context for inference, including the corpus from a file."""
        try:
            # Load the pre-trained sentence transformer model
            self.model = SentenceTransformer.load(context.artifacts["model_path"])
            
            # Load the corpus from the specified file
            corpus_file = context.artifacts["corpus_file"]
            with open(corpus_file, 'r') as file:
                self.corpus = file.read().splitlines()
            
            # Encode the corpus and convert it to a tensor
            self.corpus_embeddings = self.model.encode(self.corpus, convert_to_tensor=True)

        except Exception as e:
            raise ValueError(f"Error loading model and corpus: {e}")

    def predict(self, context, model_input, params=None):
        """Predict method to perform semantic search over the corpus."""
        
        if isinstance(model_input, pd.DataFrame):
            if model_input.shape[1] != 1:
                raise ValueError("DataFrame input must have exactly one column.")
            model_input = model_input.iloc[0, 0]
        elif isinstance(model_input, dict):
            model_input = model_input.get("sentence")
            if model_input is None:
                raise ValueError("The input dictionary must have a key named 'sentence'.")
        else:
            raise TypeError(f"Unexpected type for model_input: {type(model_input)}. Must be either a Dict or a DataFrame.")

        # Encode the query
        query_embedding = self.model.encode(model_input, convert_to_tensor=True)

        # Compute cosine similarity scores
        cos_scores = util.cos_sim(query_embedding, self.corpus_embeddings)[0]

        # Determine the number of top results to return
        top_k = params.get("top_k", 3) if params else 3  # Default to 3 if not specified

        minimum_relevancy = params.get("minimum_relevancy", 0.2) if params else 0.2  # Default to 0.2 if not specified

        # Get the top_k most similar sentences from the corpus
        top_results = np.argsort(cos_scores, axis=0)[-top_k:]

        # Prepare the initial results list
        initial_results = [(self.corpus[idx], cos_scores[idx].item()) for idx in reversed(top_results)]

        # Filter the results based on the minimum relevancy threshold
        filtered_results = [result for result in initial_results if result[1] >= minimum_relevancy]

        # If all results are below the threshold, issue a warning and return the top result
        if not filtered_results:
            warnings.warn(
                "All top results are below the minimum relevancy threshold. "
                "Returning the highest match instead.",
                UserWarning
            )
            return [initial_results[0]]
        else:
            return filtered_results


/Users/benjamin.wilson/miniconda3/envs/mlflow-dev-env/lib/python3.8/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/benjamin.wilson/miniconda3/envs/mlflow-dev-env/lib/python3.8/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## Building and Preparing the Semantic Search Corpus

In this section of our tutorial, we focus on constructing and preparing the corpus for our semantic search model. The corpus forms the backbone of our search functionality, serving as the database against which user queries are compared.

### Simulating a Real-World Use Case

In a real-world scenario, a semantic search model might need to sift through tens of thousands of blog posts stored in a large database. For the purpose of this tutorial, we are using a simplified subset to demonstrate the core principles without the complexity of handling a massive dataset. Our corpus consists of synthetic blog post entries, each composed of a title and the first two sentences of the post.

#### Key Steps in Corpus Preparation:

1. **Corpus Creation**: We create a list named `corpus`, where each element is a string representing a blog post (title plus the first two sentences).
2. **Writing to a File**: The corpus is then written to a file (`search_corpus.txt`). In a real application, this could represent extracting and preprocessing data from a larger database.

### Efficient Data Handling for Scalability

In our `SemanticSearchModel`, the corpus is encoded only once when the model is loaded for inference. This process converts the textual data into semantically meaningful embeddings using the Sentence Transformer model. These embeddings are stored in memory, which allows for efficient and rapid comparison with incoming search queries.

#### Production Considerations:

- **Storing Embeddings**: In a production environment, especially when dealing with large-scale data, storing these embeddings efficiently becomes crucial. Options include using a vector database or an in-memory database like [Redis](https://redis.com/) or [Elasticsearch](https://www.elastic.co/). These systems are optimized for handling high-dimensional data and can significantly speed up search operations.
- **Scalability**: For a use case involving millions of embeddings, a vector database or in-memory database not only provides efficient storage but also offers scalability. These systems can handle large volumes of data and support complex queries, essential for real-time semantic search applications.
- **Updating the Corpus**: In a dynamic application where new blog posts are continually added, the system would periodically update the corpus and its embeddings. This could be achieved through incremental updates or scheduled reprocessing of the entire dataset.

### Realizing the Semantic Search Concept

By setting up this corpus and discussing its practical implications, we lay the groundwork for realizing a semantic search system. The approach demonstrated here, while simplified for tutorial purposes, mirrors the foundational steps necessary in a large-scale, real-world application. It showcases how advanced NLP techniques, combined with efficient data handling and storage solutions, can be leveraged to create powerful and scalable semantic search platforms.


In [2]:
corpus = [
    "Perfecting a Sourdough Bread Recipe: The Joy of Baking. Baking sourdough bread "
    "requires patience, skill, and a good understanding of yeast fermentation. Each "
    "loaf is unique, telling its own story of the baker's journey.",
    
    "The Mars Rover's Discoveries: Unveiling the Red Planet. NASA's Mars rover has "
    "sent back stunning images and data, revealing the planet's secrets. These "
    "discoveries may hold the key to understanding Mars' history.",
    
    "The Art of Growing Herbs: Enhancing Your Culinary Skills. Growing your own "
    "herbs can transform your cooking, adding fresh and vibrant flavors. Whether it's "
    "basil, thyme, or rosemary, each herb has its own unique characteristics.",
    
    "AI in Software Development: Transforming the Tech Landscape. The rapid "
    "advancements in artificial intelligence are reshaping how we approach software "
    "development. From automation to machine learning, the possibilities are endless.",
    
    "Backpacking Through Europe: A Journey of Discovery. Traveling across Europe by "
    "backpack allows one to immerse in diverse cultures and landscapes. It's an "
    "adventure that combines the thrill of exploration with personal growth.",
    
    "Shakespeare's Timeless Influence: Reshaping Modern Storytelling. The works of "
    "William Shakespeare continue to inspire and influence contemporary literature. "
    "His mastery of language and deep understanding of human nature are unparalleled.",
    
    "The Rise of Renewable Energy: A Sustainable Future. Embracing renewable energy "
    "is crucial for achieving a sustainable and environmentally friendly lifestyle. "
    "Solar, wind, and hydro power are leading the way in this green revolution.",
    
    "The Magic of Jazz: An Exploration of Sound and Harmony. Jazz music, known for "
    "its improvisation and complex harmonies, has a rich and diverse history. It "
    "evokes a range of emotions, often reflecting the soul of the musician.",
    
    "Yoga for Mind and Body: The Benefits of Regular Practice. Engaging in regular "
    "yoga practice can significantly improve flexibility, strength, and mental "
    "well-being. It's a holistic approach to health, combining physical and spiritual "
    "aspects.",
    
    "The Egyptian Pyramids: Monuments of Ancient Majesty. The ancient Egyptian "
    "pyramids, monumental tombs for pharaohs, are marvels of architectural "
    "ingenuity. They stand as a testament to the advanced skills of ancient builders.",
    
    "Vegan Cuisine: A World of Flavor. Exploring vegan cuisine reveals a world of "
    "nutritious and delicious possibilities. From hearty soups to delectable desserts, "
    "plant-based dishes are diverse and satisfying.",

    "Extraterrestrial Life: The Endless Search. The quest to find life beyond Earth "
    "continues to captivate scientists and the public alike. Advances in space "
    "technology are bringing us closer to answering this age-old question.",

    "The Art of Plant Pruning: Promoting Healthy Growth. Regular pruning is essential "
    "for maintaining healthy and vibrant plants. It's not just about cutting back, but "
    "understanding each plant's growth patterns and needs.",

    "Cybersecurity in the Digital Age: Protecting Our Data. With the rise of digital "
    "technology, cybersecurity has become a critical concern. Protecting sensitive "
    "information from cyber threats is an ongoing challenge for individuals and "
    "businesses alike.",

    "The Great Wall of China: A Historical Journey. Visiting the Great Wall offers "
    "more than just breathtaking views; it's a journey through history. This ancient "
    "structure tells stories of empires, invasions, and human resilience.",

    "Mystery Novels: Crafting Suspense and Intrigue. A great mystery novel captivates "
    "the reader with intricate plots and unexpected twists. It's a genre that combines "
    "intellectual challenge with entertainment.",

    "Conserving Endangered Species: A Global Effort. Protecting endangered species "
    "is a critical task that requires international collaboration. From rainforests to "
    "oceans, every effort counts in preserving our planet's biodiversity.",

    "Emotions in Classical Music: A Symphony of Feelings. Classical music is not just "
    "an auditory experience; it's an emotional journey. Each composition tells a story, "
    "conveying feelings from joy to sorrow, tranquility to excitement.",

    "CrossFit: A Test of Strength and Endurance. CrossFit is more than just a fitness "
    "regimen; it's a lifestyle that challenges your physical and mental limits. It "
    "combines various disciplines to create a comprehensive workout.",

    "The Renaissance: An Era of Artistic Genius. The Renaissance marked a period of "
    "extraordinary artistic and scientific achievements. It was a time when creativity "
    "and innovation flourished, reshaping the course of history.",

    "Exploring International Cuisines: A Culinary Adventure. Discovering international "
    "cuisines is an adventure for the palate. Each dish offers a glimpse into the "
    "culture and traditions of its origin.",

    "Astronaut Training: Preparing for the Unknown. Becoming an astronaut involves "
    "rigorous training to prepare for the extreme conditions of space. It's a journey "
    "that tests both physical endurance and mental resilience.",

    "Sustainable Gardening: Nurturing the Environment. Sustainable gardening is not "
    "just about growing plants; it's about cultivating an ecosystem. By embracing "
    "environmentally friendly practices, gardeners can have a positive impact on the "
    "planet.",

    "The Smartphone Revolution: Changing Communication. Smartphones have transformed "
    "how we communicate, offering unprecedented connectivity and convenience. This "
    "technology continues to evolve, shaping our daily interactions.",

    "Experiencing African Safaris: Wildlife and Wilderness. An African safari is an "
    "unforgettable experience that brings you face-to-face with the wonders of "
    "wildlife. It's a journey that connects you with the raw beauty of nature.",

    "Graphic Novels: A Blend of Art and Story. Graphic novels offer a unique medium "
    "where art and narrative intertwine to tell compelling stories. They challenge "
    "traditional forms of storytelling, offering visual and textual richness.",

    "Addressing Ocean Pollution: A Call to Action. The increasing levels of pollution "
    "in our oceans are a pressing environmental concern. Protecting marine life and "
    "ecosystems requires concerted global efforts.",

    "The Origins of Hip Hop: A Cultural Movement. Hip hop music, originating from the "
    "streets of New York, has grown into a powerful cultural movement. Its beats and "
    "lyrics reflect the experiences and voices of a community.",

    "Swimming: A Comprehensive Workout. Swimming offers a full-body workout that is "
    "both challenging and refreshing. It's an exercise that enhances cardiovascular "
    "health, builds muscle, and improves endurance.",

    "The Fall of the Berlin Wall: A Historical Turning Point. The fall of the Berlin "
    "Wall was not just a physical demolition; it was a symbol of political and social "
    "change. This historic event marked the end of an era and the beginning of a new "
    "chapter in world history."
]

# Write the corpus to a file
corpus_file = '/tmp/search_corpus.txt'
with open(corpus_file, 'w') as file:
    for sentence in corpus:
        file.write(sentence + '\n')

## Model Preparation and Configuration in MLflow

In this section, we delve into the setup of our Sentence Transformer model and its integration with MLflow. This step is crucial for preparing the model for deployment and ensuring it can be used effectively in a practical setting.

### Loading and Saving the Sentence Transformer Model

1. **Model Initialization**: 
    - We begin by loading a pre-trained sentence transformer model using the `SentenceTransformer` class from the `sentence_transformers` library. 
    - The model chosen is `"all-MiniLM-L6-v2"`, which is a compact and efficient transformer model known for its balance between performance and speed, making it ideal for semantic search tasks.

2. **Model Storage**:
    - The model is then saved to a directory (`/tmp/search_model`). This step is essential for creating a portable version of the model that can be loaded by MLflow later for deployment. In production, the location to store the model would be to a permanent location that the production deployment service has access to.
    - For the purposes of this tutorial, saving the model locally also allows for version control and easy management of the model artifact.

### Preparing Model Artifacts and Signature

1. **Artifacts Dictionary**:
    - We create a dictionary named `artifacts`, which contains paths to the model and the corpus file. 
    - This dictionary is crucial as it tells MLflow where to find the necessary components for the model to function correctly during inference.

2. **Input Example and Test Output**:
    - An input example (`input_example`) is defined, simulating a typical query that the model is expected to process. This helps in understanding the model's input structure.
    - Similarly, `test_output` provides a sample of the expected output format. In our case, it's a list of matched sentences.

3. **Model Signature**:
    - The `infer_signature` function from MLflow is used to automatically generate a model signature based on the provided input and output examples.
    - The signature describes the model's input and output schema, providing valuable metadata about the types and shapes of data the model expects and produces.
    - We also include parameters `top_k` and `minimum_relevancy` in the signature to explicitly define the model's expected behavior regarding the number of results and their relevancy threshold.

### Importance of the Model Signature

- The signature plays a vital role in model deployment and usage. It ensures consistency between the model training environment and the deployment environment, reducing the chances of errors due to mismatched data formats.
- It also aids in the interpretability of the model, making it clear to users how to interact with the model and what kind of results to expect.

### Conclusion

This setup process is a critical part of model preparation in MLflow. It involves not only loading from the Hugging Face Hub and locally saving the model but also meticulously preparing all associated artifacts and defining a clear and informative model signature. This comprehensive approach ensures that the model is ready for deployment, with all its dependencies and operational requirements clearly specified.


In [3]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Create an input example DataFrame
input_example = ["Something I want to find matches for."]

# Save the model in the /tmp directory
model_directory = "/tmp/search_model"
model.save(model_directory)

artifacts = {
    "model_path": model_directory,
    "corpus_file": corpus_file
}

# Generate test output for signature
test_output = ["match 1", "match 2", "match 3"]

# Define the signature associated with the model
signature = infer_signature(input_example, test_output, params={"top_k": 3, "minimum_relevancy": 0.2})

signature

inputs: 
  [string]
outputs: 
  [string]
params: 
  ['top_k': long (default: 3), 'minimum_relevancy': double (default: 0.2)]

### Setting the tracking server and creating an experiment

In order to view the results in our tracking server (for the purposes of this tutorial, we’ve started a local tracking server at this url)

We can start an instance of the MLflow server locally by running the following from a terminal to start the tracking server:

``` bash
mlflow server --host 127.0.0.1 --port 8080
```

With the server started, the following code will ensure that all experiments, runs, models, parameters, and metrics that we log are being tracked within that server instance (which also provides us with the MLflow UI when navigating to that url address in a browser).

After setting the tracking url, we create a new MLflow Experiment to store the run we’re about to create in.

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

mlflow.set_experiment("Semantic Similarity")

<Experiment: artifact_location='mlflow-artifacts:/413386080563320984', creation_time=1700272457800, experiment_id='413386080563320984', last_update_time=1700272457800, lifecycle_stage='active', name='Semantic Similarity', tags={}>

## Logging the Model with MLflow

After setting up the model and its necessary components, the next critical step in our tutorial is to log the model using MLflow. This process registers the model in the MLflow tracking system, making it manageable and deployable through the MLflow framework.

### Starting an MLflow Run

1. **Context Management**: 
    - We use the `with mlflow.start_run() as run:` statement to initiate an MLflow run. This statement creates a new run in MLflow's tracking system, which will record metrics, parameters, and the model itself.
    - Using `with` ensures that the run is properly closed after all operations within the block are completed, maintaining clean and consistent tracking.

### Logging the Model

2. **Model Logging**: 
    - The `mlflow.pyfunc.log_model` function is used to log our custom Python function model, `SemanticSearchModel`, in MLflow.
    - Key arguments in `log_model` function:
        - `"semantic_search"`: The name given to the logged model.
        - `python_model=SemanticSearchModel()`: Specifies the instance of our custom model class.
        - `input_example=input_example`: Provides a sample input for the model, used for documentation and to help MLflow understand the model's input format.
        - `signature=signature`: The model signature, as previously defined, describing input and output schemas.
        - `artifacts=artifacts`: The artifacts dictionary, which includes paths to the model and the corpus file.
        - `pip_requirements=["sentence_transformers", "numpy"]`: Specifies Python package requirements needed for the model to run. This ensures that the deployment environment is correctly set up with the necessary dependencies.

### Outcome of Model Logging

- **Model Registration**: The model, along with its artifacts, signature, and requirements, is now registered in MLflow. It is stored in a way that is easily accessible and deployable.
- **Reproducibility and Traceability**: By logging the model in MLflow, we ensure reproducibility of results and maintain traceability of the model's version and its associated data.

### Conclusion

Logging the model in MLflow is a crucial step that bridges the gap between model development and deployment. It encapsulates the model and its dependencies in a format that is ready for production use, ensuring that the model can be deployed consistently regardless of the deployment environment.


In [5]:
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        "semantic_search",
        python_model=SemanticSearchModel(),
        input_example=input_example,
        signature=signature,
        artifacts=artifacts,
        pip_requirements=["sentence_transformers", "numpy"],
    )

2023/11/20 11:59:40 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


/Users/benjamin.wilson/miniconda3/envs/mlflow-dev-env/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Model Inference and Prediction Demonstration

Having logged our semantic search model with MLflow, we now move to the crucial phase of loading and using the model for making predictions. This step demonstrates the model's practical application in responding to user queries.

### Loading the Model for Inference

1. **Model Loading**: 
    - We use `mlflow.pyfunc.load_model` to load the model for inference. This function takes the model's URI, which uniquely identifies the model in the MLflow tracking system.
    - The loaded model, `loaded_dynamic`, is now an instance of our `SemanticSearchModel` and is ready to process queries.

### Making a Prediction

2. **Running a Query**:
    - To demonstrate the model's functionality, we pass a sample query: `"I'd like some ideas for a meal to cook."` This query is representative of the kind of user input the model is designed to handle.
    - The `predict` method of `loaded_dynamic` is called with the query, triggering the semantic search process over the corpus.

### Understanding the Prediction Output

- **Output Format**:
    - The output is a list of tuples, where each tuple contains a matched corpus entry and its corresponding cosine similarity score.
    - The cosine similarity score represents how relevant each entry is to the query, with higher scores indicating greater relevance.

3. **Example Results**:
    - The model returns entries related to cooking and culinary skills, showcasing its ability to understand the semantic context of the query.
    - The results include matches related to international cuisines, vegan cooking, and herb gardening, each with a relevance score:
        - International cuisines: 0.4385
        - Vegan cuisine: 0.3469
        - Growing herbs: 0.2269
    - These scores indicate the model's assessment of how closely each blog post matches the user's query.

### Conclusion

This demonstration highlights the model's capability to interpret a user's query semantically and return relevant blog post suggestions. The results show the effectiveness of the Sentence Transformers in encoding semantic meaning and the power of cosine similarity in matching queries to corpus entries. It exemplifies the potential of our semantic search model in real-world applications, such as recommendation systems or knowledge retrieval platforms.

In [6]:
loaded_dynamic = mlflow.pyfunc.load_model(model_info.model_uri)

loaded_dynamic.predict(["I'd like some ideas for a meal to cook."])

2023/11/20 11:59:41 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


[('Exploring International Cuisines: A Culinary Adventure. Discovering international cuisines is an adventure for the palate. Each dish offers a glimpse into the culture and traditions of its origin.',
  0.43857115507125854),
 ('Vegan Cuisine: A World of Flavor. Exploring vegan cuisine reveals a world of nutritious and delicious possibilities. From hearty soups to delectable desserts, plant-based dishes are diverse and satisfying.',
  0.34688490629196167),
 ("The Art of Growing Herbs: Enhancing Your Culinary Skills. Growing your own herbs can transform your cooking, adding fresh and vibrant flavors. Whether it's basil, thyme, or rosemary, each herb has its own unique characteristics.",
  0.22686949372291565)]

## Advanced Query Handling with Customizable Parameters and Warning Mechanism

Our semantic search model not only offers customizable search parameters but also incorporates a warning mechanism for cases where the results do not meet certain criteria. This feature enhances the model's robustness and usability, ensuring that users are informed about the quality of the search results.

### Executing a Customized Prediction with Warnings

1. **Customized Query with Challenging Parameters**:
    - We issue a query: `"Latest stories on computing"` with specific parameters `{"top_k": 10, "minimum_relevancy": 0.4}`.
    - This query, coupled with a high relevancy threshold, creates a challenging scenario for the model, testing its ability to discern highly relevant content.

2. **Triggering the Warning**:
    - Due to the high `minimum_relevancy` threshold (0.4), the model may not find enough results that meet this criterion.
    - When this occurs, the model triggers a `UserWarning`, alerting that all top results are below the minimum relevancy threshold.
    - This warning mechanism is vital for user feedback, indicating that the search criteria might be too restrictive or that the corpus may not contain sufficiently relevant content for the specific query.

### Understanding the Model's Response

- **Result in Challenging Scenarios**:
    - Despite the stringent criteria, the model still provides the best available match. In our example, the highest relevant match relates to AI in software development, with a score of 0.2534.
    - This result, although below the threshold, is the closest match to the query based on the available corpus data.

### Implications and Best Practices

- **Balancing Relevancy and Coverage**:
    - The `minimum_relevancy` parameter should be set considering the nature of the corpus and the typical use cases. Setting it too high may lead to frequent warnings and fewer results.
    - Understanding the balance between relevancy and the breadth of search results is crucial for optimizing user experience.

- **User Feedback for Corpus Improvement**:
    - Warnings can provide valuable feedback for refining the corpus. If certain queries consistently trigger warnings, it may indicate gaps in the corpus that need to be addressed.

### Conclusion

This advanced demonstration with a warning mechanism highlights the model's capability to handle complex search scenarios and provide meaningful feedback. It underscores the importance of tuning search parameters appropriately and using user feedback to continually improve the search system. Such features make our semantic search model not just a tool for retrieval but also a dynamic system capable of adapting and improving over time.

In [7]:
loaded_dynamic.predict(["Latest stories on computing"], params={"top_k": 10, "minimum_relevancy": 0.4})

/var/folders/cd/n8n0rm2x53l_s0xv_j_xklb00000gp/T/ipykernel_79034/2125590310.py:65: UserWarning: All top results are below the minimum relevancy threshold. Returning the highest match instead.
  warnings.warn(


[('AI in Software Development: Transforming the Tech Landscape. The rapid advancements in artificial intelligence are reshaping how we approach software development. From automation to machine learning, the possibilities are endless.',
  0.2533860206604004)]

## Conclusion: Crafting Custom Logic with MLflow's PythonModel

As we wrap up this tutorial, let's reflect on the key learnings and the powerful capabilities of MLflow's `PythonModel` in crafting custom logic for real-world applications, particularly when integrating advanced libraries like `sentence-transformers`.

### Key Takeaways

1. **Flexibility of PythonModel**:
    - The `PythonModel` in MLflow offers unparalleled flexibility in defining custom logic. Throughout this tutorial, we leveraged this to build a semantic search model tailored to our specific requirements.
    - This flexibility proves invaluable when dealing with complex use cases that go beyond standard model implementations.

2. **Integration with Sentence Transformers**:
    - We seamlessly integrated the `sentence-transformers` library within our MLflow model. This demonstrated how advanced NLP capabilities can be embedded within custom models to handle sophisticated tasks like semantic search.
    - The use of transformer models for generating embeddings showcased how cutting-edge NLP techniques could be applied in practical scenarios.

3. **Customization and User Experience**:
    - Our model not only performed the core task of semantic search but also allowed for customizable search parameters (`top_k` and `minimum_relevancy`). This level of customization is crucial for aligning the model's output with varying user needs.
    - The inclusion of a warning mechanism further enriched the model by providing valuable feedback, enhancing the user experience.

4. **Real-World Application and Scalability**:
    - While our tutorial focused on a controlled dataset, the principles and methodologies apply to much larger, real-world datasets. The discussion around using vector databases and in-memory databases like Redis or Elasticsearch for scalability highlighted how the model could be adapted for large-scale applications.

### Empowering Real-World Applications

- The combination of MLflow's `PythonModel` and advanced libraries like `sentence-transformers` simplifies the creation of sophisticated, real-world applications. 
- The ability to encapsulate complex logic, manage dependencies, and ensure model portability makes MLflow an invaluable tool in the modern data scientist's toolkit.

### Moving Forward

- As we conclude, remember that the journey doesn't end here. The concepts and techniques explored in this tutorial lay the groundwork for further exploration and innovation in the field of NLP and beyond.
- We encourage you to take these learnings, experiment with your datasets, and continue pushing the boundaries of what's possible with MLflow and advanced NLP technologies.

Thank you for joining us on this enlightening journey through semantic search with Sentence Transformers and MLflow!
